<a href="https://colab.research.google.com/github/kynemre/DataMining/blob/main/04_D%C3%BCnya_Mutluluk_Endeksi_Verilerinin_G%C3%B6rselle%C5%9Ftirilmesi_ve_Yorumlanmas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dünya Mutluluk Endeksi

<p><img src="https://user-images.githubusercontent.com/46110671/118335767-081f2200-b519-11eb-98dc-33042ec55956.jpg" height=450></p><hr><br>

<p>Mutluluğun belirleyicilerini saptamaya yönelik çalışmalar, farklı bakış açılarına göre değişik şekillerde sınıflandırılmıştır. Günümüzde iktisat bilimiyle de ilişkilendirilen mutluluk kavramı ve mutluluğu ölçmeye dayalı araştırmalar, mutluluk ekonomisi çerçevesinde ele alınmaktadır.<sup><a href="https://dergipark.org.tr/tr/download/article-file/392237">[Ref]</a></sup>
</p>
<p>
Mutluluk Endeksinin tanımı, Bhutan Gayri Safi Ulusal Mutluluk Endeksinden kaynaklanmaktadır. 1972'de Bhutan'da mutluluk zenginlik, rahatlık ve ekonomik büyüme gibi faktörlerle ölçeklendirilmeye başlandı. Birden çok ölçülebilir faktöre dayanarak mutluluk için bir endeks oluşturuldu ve o zamandan beri bu endeks takip edildi.<sup><a href="https://www.matematiksel.org/zenginligi-halkinin-mutlulugu-ile-olcen-ulke-bhutan/">[Ref]</a></sup></p>
<p>Günümüzde mutluluk indeksi yerine <b>Yaşam Merdiveni (Life Ladder)</b> terimi kullanılmaktadır.</p>

#Veri Setinin İçe Aktarılması

<h3>Veri Setini İndirilmesi ve ZIP Dosyasını Açılması</h3>

In [ ]:
adress = 'https://raw.githubusercontent.com/kynemre/DataMining/main/data/Mutluluk%20Endeksi.zip';
!wget -q -O veri.zip "{adress}";#Verileri indirdik
!unzip -q veri.zip
!rm veri.zip ##Zip dosyasini sildik

<h3>Kütüphanelerin İçe Aktarılması<h3>

In [ ]:
!pip install pycountry-convert
!pip install raceplotly

In [ ]:
import pandas as pd #Dosya okuma ve veri işlemleri
import numpy as np
from tqdm import tqdm #İlerleme çubuğu
import plotly.express as px #Veri görselleştirme
from pycountry_convert import (country_name_to_country_alpha2,
                               country_alpha2_to_continent_code)
from raceplotly.plots import barplot
from plotly.subplots import make_subplots

# Yıllara Göre Analiz

<h3>Veri Setini Yükleme</h3>
<p>
Veri setimizi yükledik ve 2010-2020 yılları arasındaki verileri aldık.
</p>

In [ ]:
data_years = pd.read_csv("/content/world-happiness-report.csv")
data_years = data_years[data_years['year'] >= 2010].sort_values('year').reset_index(drop=True)
data_years.head(3)

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2010,4.758,7.647,0.539,51.6,0.600,0.121,0.707,0.618,0.275
1,Greece,2010,5.840,10.433,0.868,71.5,0.484,-0.303,0.954,0.634,0.292
2,Ecuador,2010,5.838,9.244,0.839,66.8,0.723,-0.063,0.806,0.826,0.220


<h3>NaN Değerleri Temizleme</h3>
<p>
Tanımsız değerleri kolonların ortalaması ile değiştirdik.
</p>

In [ ]:
data_years[data_years.isna().any(axis=1)].head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
3,Morocco,2010,4.383,8.746,NaN,63.5,0.663,-0.162,0.900,NaN,NaN
22,Jordan,2010,5.570,9.317,0.918,64.7,0.788,-0.046,NaN,0.643,0.343
35,Saudi Arabia,2010,6.307,10.693,0.880,64.4,0.678,-0.033,NaN,0.645,0.297
71,Uzbekistan,2010,5.095,8.445,0.903,63.2,NaN,-0.030,0.519,0.776,0.152
72,Somaliland region,2010,4.657,NaN,0.829,NaN,0.820,NaN,0.471,0.769,0.083


In [ ]:
data_years.describe()

,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
count,1507.000000,1507.000000,1473.000000,1497.000000,1459.00000,1486.000000,1454.000000,1415.000000,1490.000000,1494.000000
mean,2014.909754,5.472569,9.410657,0.811536,63.85548,0.753607,-0.002649,0.741493,0.709411,0.274310
std,3.057845,1.118708,1.143646,0.117441,7.22372,0.136876,0.161257,0.186807,0.107737,0.088024
min,2010.000000,2.375000,6.635000,0.290000,32.30000,0.304000,-0.335000,0.047000,0.322000,0.083000
25%,2012.000000,4.644000,8.506000,0.748000,58.90000,0.661000,-0.118000,0.685000,0.625000,0.208000
50%,2015.000000,5.439000,9.510000,0.832000,65.60000,0.773000,-0.030000,0.797000,0.718000,0.266000
75%,2018.000000,6.290500,10.373000,0.905000,68.75000,0.862000,0.088000,0.866000,0.798750,0.327000
max,2020.000000,7.889000,11.648000,0.987000,77.10000,0.985000,0.698000,0.983000,0.944000,0.705000


In [ ]:
data_years.fillna(data_years.mean(), inplace=True)
data_years.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1507 entries, 0 to 1506
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country name                      1507 non-null   object 
 1   year                              1507 non-null   int64  
 2   Life Ladder                       1507 non-null   float64
 3   Log GDP per capita                1507 non-null   float64
 4   Social support                    1507 non-null   float64
 5   Healthy life expectancy at birth  1507 non-null   float64
 6   Freedom to make life choices      1507 non-null   float64
 7   Generosity                        1507 non-null   float64
 8   Perceptions of corruption         1507 non-null   float64
 9   Positive affect                   1507 non-null   float64
 10  Negative affect                   1507 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 129.6+ KB


<h3>Eksik Verileri Ülkeleri Temizleme</h3>
<p>
Yıl eksiği olan ülkeleri sildik.
</p>

In [ ]:
countries = dict(data_years["Country name"].value_counts())
data_years = data_years[data_years['Country name'].isin([name for name, v in countries.items() if v==11])]
data_years["Country name"].value_counts()

Zimbabwe           11
Mexico             11
Ukraine            11
North Macedonia    11
Brazil             11
                   ..
Thailand           11
Ghana              11
Sweden             11
Cyprus             11
Moldova            11
Name: Country name, Length: 75, dtype: int64

<h3>Ülkelerin Bulundukları Kıtaları Alma</h3>
<p>
Ülkelerin bulundukları kıtaları veri setimize ekledik.
</p>

In [ ]:
continents = {
    'AF':	'Africa',
    'NA':	'North America',
    'OC':	'Oceania',
    'AN':	'Antarctica',
    'AS':	'Asia',
    'EU':	'Europe',
    'SA':	'South America'
}
continent = []
for name in data_years['Country name']:
    try:
        continent.append(continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(name))])
    except:
        continent.append("Unknown")

data_years["Continent"] = continent
data_years = data_years.round(2)
data_years.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Continent
1,Greece,2010,5.84,10.43,0.87,71.5,0.48,-0.30,0.95,0.63,0.29,Europe
2,Ecuador,2010,5.84,9.24,0.84,66.8,0.72,-0.06,0.81,0.83,0.22,South America
4,Cameroon,2010,4.55,8.04,0.76,48.3,0.79,0.00,0.88,0.61,0.27,Africa
5,United Kingdom,2010,7.03,10.65,0.96,71.3,0.84,0.40,0.59,0.86,0.18,Europe
7,Montenegro,2010,5.46,9.72,0.80,67.1,0.55,-0.21,0.76,0.60,0.41,Europe


# Veri Görselleştrime

In [ ]:
fig = px.histogram(data_years,
                   x='Life Ladder',
                   title="Ülkelerin Mutluluk İndekseleri Histogramı",
                   labels={
                       'Country name':'Ülke Adı',
                        'Life Ladder':'Yaşam Merdiveni'},
                    color_discrete_sequence=['#209910']) #Sütun grafiğinin çizilmesi
fig.show()

In [ ]:
temp_data = data_years[(data_years['year']==2020)].sort_values('Life Ladder')[::-1]
fig = px.bar(temp_data,
             x='Country name',
             y='Life Ladder',
             title="2020 Yılında Ülkelerin Mutluluk İndekseleri",
             range_y=(temp_data['Life Ladder'].min()-0.5,temp_data['Life Ladder'].max()+.12),
             labels={
                 'Country name':'Ülke Adı',
                 'Life Ladder':'Yaşam Merdiveni'},
             color_discrete_sequence=['#209910']) #Sütun grafiğinin çizilmesi
fig.update_xaxes(tickangle=-60)
fig.show()
del temp_data

In [ ]:
temp_data = data_years[(data_years['year']==2020)][['Country name', 'year', 'Life Ladder']].sort_values('Country name')
temp_data['Diff'] = np.array(temp_data['Life Ladder']) - np.array(data_years[(data_years['year']==2010)].sort_values('Country name')['Life Ladder'])
temp_data = temp_data[['Country name', 'Diff']].sort_values('Diff')

fig = px.bar(temp_data,
             x='Country name',
             y='Diff',
             title="2010 Yılından 2020 Yılına Ülkelerin Mutluluk İndekslerindeki Değişim",
             range_y=(-abs(temp_data['Diff']).max()-0.1,abs(temp_data['Diff']).max()+.1),
             labels={'Country name':'Ülke Adı',
                     'Diff':'Değişim Miktarı'},            
             color_discrete_sequence=['#209910']) #Sütun grafiğinin çizilmesi
fig.update_xaxes(tickangle=-60)
fig.show()
del temp_data

In [ ]:
temp_data = data_years[['Continent', 'Life Ladder']].groupby('Continent').mean().reset_index().round(2)
fig = px.bar(temp_data,
             x='Continent',
             y='Life Ladder',
             title="Kıtaların Ortalama Mutluluk İndeksleri",
             range_y=(0,temp_data['Life Ladder'].max()+1),
             labels={'Continent':'Kıta Adı',
                     'Life Ladder':'Mutluluk İndeksi'},            
             color_discrete_sequence=['#209910']) #Sütun grafiğinin çizilmesi
fig.update_xaxes(tickangle=-60)
fig.show()
del temp_data

In [ ]:
colorList = ['#370617', '#6a040f', '#9d0208','#d00000', '#dc2f02', '#e85d04',
             '#e85d04', '#f48c06', '#faa307', '#ffba08', '#ffff09']
fig = px.choropleth(data_years[data_years['year']==2020], 
                    locations="Country name",
                    locationmode="country names",
                    color="Life Ladder", 
                    hover_name="Country name",
                    hover_data=data_years.columns[2:-1],
                    title="2020 Yılında Ülkelerin Mutluluk İndekseleri",
                    color_continuous_scale=colorList)
fig.show()

In [ ]:
temp_data = data_years[['Country name', 'Life Ladder']].groupby('Country name').mean().reset_index().round(2)
fig = px.choropleth(temp_data, 
                    locations="Country name",
                    locationmode="country names",
                    color="Life Ladder", 
                    hover_name="Country name",
                    title="Ülkelerin Yıllara Göre Ortalama Mutluluk İndekseleri",
                    color_continuous_scale=colorList)
fig.show()
del temp_data

In [ ]:
fig = px.choropleth(data_years.sort_values("year"), 
                    locations="Country name",
                    locationmode="country names",
                    color="Life Ladder", 
                    hover_name="Country name",
                    hover_data=data_years.columns[2:-1],
                    animation_frame="year",
                    range_color=(data_years["Life Ladder"].min(),
                                 data_years["Life Ladder"].max()),
                    title="Yıllara Göre Ülkelerin Mutluluk İndekseleri",
                    color_continuous_scale=px.colors.sequential.Hot)
fig.show()

In [ ]:
my_raceplot = barplot(data_years,
                      item_column='Country name',
                      value_column='Life Ladder',
                      time_column='year')

my_raceplot.plot(item_label = 'Country',
                 value_label = 'Life Ladder',
                 frame_duration = 1500,
                 date_format = '%Y')

In [ ]:
fig = px.scatter_matrix(data_years[data_years['year']==2020], 
                        dimensions=['Life Ladder', 'Log GDP per capita',
                                    'Social support', 'Healthy life expectancy at birth',
                                    'Freedom to make life choices', 'Generosity',
                                    'Perceptions of corruption'],
                        title="Niteliklerin Dağılım Grafikleri",
                        labels={'Life Ladder':'LL',
                                'Log GDP per capita':'GDP',
                                'Social support':'Support',
                                'Healthy life expectancy at birth':'Healty Life',
                                'Freedom to make life choices':'Freedom',
                                'Generosity':'Generosity',
                                'Perceptions of corruption':'Perceptions'},
                        width=1200, height=800)
fig.update_traces(diagonal_visible=False,
                  showupperhalf=False)
fig.show()

In [ ]:
fig = px.scatter(data_years[data_years['year']==2020].sort_values('Continent'),
                 x="Life Ladder",
                 y="Log GDP per capita",
                 color="Continent",
                 hover_name="Country name")
fig.update_traces(marker=dict(size=12,
                              opacity=0.9,
                              line=dict(width=0.5,
                                        color='white')),
                  selector=dict(mode='markers'))
fig['data'][0]['showlegend']=True
for i,name in enumerate(list(data_years.sort_values('Continent')['Continent'].unique())):
    fig['data'][i]['name']= name

fig.show()

In [ ]:
fig = px.scatter(data_years, 
                 x="Log GDP per capita", 
                 y="Life Ladder", 
                 animation_frame="year", 
                 animation_group="Country name",
                 color="Continent",
                 hover_name="Country name",
                 facet_col="Continent",
                 range_x=(data_years['Log GDP per capita'].min()-0.5,
                          data_years['Log GDP per capita'].max()+0.5),
                 range_y=(data_years['Life Ladder'].min()-0.5,
                          data_years['Life Ladder'].max()+0.5))

fig.update_traces(marker=dict(size=12,
                              opacity=0.9,
                              line=dict(width=0.5,
                                        color='white')),
                  selector=dict(mode='markers'))

fig['data'][0]['showlegend']=True
for i,name in enumerate(list(data_years['Continent'].unique())):
    fig['data'][i]['name'] = name
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()